In [1]:
from utils.general import *
from utils.video_processing import *
from utils.chat_api import *
from prompts import *

import numpy as np
from tqdm import tqdm
import json

In [2]:
def answer_question(video_clips, qa, model):
    question = qa["question"]
    gt = qa["answer"]

    inputs = [
        [
            {"type": "video_base64/mp4", "content": clip},
            {"type": "text", "content": prompt_baseline_answer_clipwise_extract},
            {"type": "text", "content": f"Question: {question}"},
            {"type": "text", "content": "Extracted information:"},
        ]
        for clip in video_clips
    ]
    messages = [generate_messages(input) for input in inputs]
    responses = parallel_get_response(model, messages)

    extracted_information = [
        response for response in responses[0] if not response.lower().startswith("none")
    ]
    qa["extracted_information"] = extracted_information
    if len(extracted_information) == 0:
        answer = "Unanswerable."
    else:
        input = [
            {"type": "text", "content": prompt_baseline_answer_clipwise_summarize},
            {"type": "text", "content": f"Question: {question}"},
            {
                "type": "text",
                "content": f"Extracted information: {extracted_information}",
            },
            {"type": "text", "content": "Answer:"},
        ]
        messages = generate_messages(input)
        model = "gpt-4o-2024-11-20"
        response = get_response_with_retry(model, messages)
        answer = response[0]

    qa["answer_baselines"] = answer
    return qa


def verify_answers(qas):
    inputs = [
        [
            {
                "type": "text",
                "content": json.dumps(qa),
            },
            {
                "type": "text",
                "content": prompt_benchmark_verify_answer,
            },
            {
                "type": "text",
                "content": "Now answer if the answer from the baseline is correct or not:",
            },
        ]
        for qa in qas
    ]
    messages = [generate_messages(input) for input in inputs]
    model = "gpt-4o-2024-11-20"
    responses = parallel_get_response(model, messages)

    results = responses[0]

    # calculate the accuracy of the answers
    correct = 0
    for result in results:
        if result.lower().startswith("yes"):
            correct += 1
    accuracy = correct / len(results)

    return accuracy, results


def process_video(video, interval_seconds):
    """Process video segments at specified intervals with given fps.

    Args:
        video_path (str): Path to the video file
        interval_seconds (float): Time interval between segments in seconds
        fps (float): Frames per second to extract from each segment

    Returns:
        None
    """

    video_path = video["path"]
    qa_list = video["qa_list"]

    video_info = get_video_info(video_path)
    print(video_info)

    clips = []

    # Process each interval
    for start_time in np.arange(0, video_info["duration"], interval_seconds):
        if start_time >= video_info["duration"]:
            break

        base64_video, _, _ = process_video_clip(
            video_path, start_time, interval_seconds
        )
        clips.append(base64_video)

    model = "gemini-1.5-pro-002"
    qpm = config[model]["qpm"]
    qa_batch_size = qpm // len(clips)
    qa_batches = [
        qa_list[i : i + qa_batch_size] for i in range(0, len(qa_list), qa_batch_size)
    ]

    answered_qa_list = []

    for qa_batch in qa_batches:
        # parallel question answering with multiple threads
        with ThreadPoolExecutor(max_workers=len(qa_batch)) as executor:
            futures = [
                executor.submit(answer_question, clips, qa, model) for qa in qa_batch
            ]
            for future in as_completed(futures):
                qa = future.result()
                answered_qa_list.append(qa)

    video["qa_list"] = answered_qa_list
    return video

In [3]:
import json

with open("data/annotations/video_list_CZ_modified.json", "r") as f:
    data = json.load(f)

for video in tqdm(data):
    # print(video)
    video = process_video(video, 180)
    with open("data/annotations/video_list_CZ_answer_clipwise_0320.json", "w") as f:
        json.dump(data, f, indent=4)

  0%|          | 0/50 [00:00<?, ?it/s]

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42', 'creation_time': '2025-02-17T22:45:05.000000Z', 'encoder': 'Google'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 360], 'bitrate': 129, 'fps': 23.976023976023978, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'creation_time': '2025-02-17T22:45:05.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 02/17/2025.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'creation_time': '2025-02-17T22:45:05.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 02/17/2025.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 1744.63, 'bitrate': 261

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42', 'creation_time': '2025-02-17T22:45:05.000000Z', 'encoder': 'Google'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 360], 'bitrate': 129, 'fps': 23.976023976023978, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'creation_time': '2025-02-17T22:45:05.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 02/17/2025.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'creation_time': '2025-02-17T22:45:05.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 02/17/2025.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 1744.63, 'bitrate': 261

  2%|▏         | 1/50 [08:43<7:07:32, 523.52s/it]

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42', 'encoder': 'Google'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 360], 'bitrate': 456, 'fps': 23.976023976023978, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 44100, 'bitrate': 95, 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 1639.44, 'bitrate': 554, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(Main)', 'video_size': [640, 360], 'video_bitrate': 456, 'video_

...

/usr/local/lib/python3.9/dist-packages/moviepy/video/io/ffmpeg_reader.py:178: UserWarning: In file data/videos/raw/360p/oTzSzfZGV2E.mp4, 691200 bytes wanted but 0 bytes read at frame index 36424 (out of a total 36425 frames), at time 1456.96/1457.03 sec. Using the last valid frame instead.
  warnings.warn(


{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42', 'encoder': 'Google'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 360], 'bitrate': 458, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 44100, 'bitrate': 96, 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 1457.03, 'bitrate': 557, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(Main)', 'video_size': [640, 360], 'video_bitrate': 458, 'video_fps': 25.0, 'd

/usr/local/lib/python3.9/dist-packages/moviepy/video/io/ffmpeg_reader.py:178: UserWarning: In file data/videos/raw/360p/oTzSzfZGV2E.mp4, 691200 bytes wanted but 0 bytes read at frame index 36425 (out of a total 36425 frames), at time 1457.00/1457.03 sec. Using the last valid frame instead.
  warnings.warn(
 98%|█████████▊| 49/50 [7:14:17<09:17, 557.97s/it]

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42', 'creation_time': '2018-10-10T20:23:57.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 360], 'bitrate': 151, 'fps': 23.976023976023978, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'creation_time': '2018-10-10T20:23:57.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 10/10/2018.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 44100, 'bitrate': 125, 'metadata': {'Metadata': '', 'creation_time': '2018-10-10T20:23:57.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 10/10/2018.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 1243.71, 'bitrate': 281, 'start': 0.0, 'defa

100%|██████████| 50/50 [7:21:43<00:00, 530.06s/it]


In [4]:
with open("data/annotations/video_list_CZ_answer_clipwise_0320.json", "r") as f:
    video_list = json.load(f)

qas = []
for video in video_list:
    qa_list = [
        {
            "question": qa["question"],
            "ground truth answer": qa["answer"],
            "baseline answer": qa["answer_baselines"],
        }
        for qa in video["qa_list"]
    ]
    qas.extend(qa_list)

acc, results = verify_answers(qas)

print(acc)

idx = 0
for video in video_list:
    for qa in video["qa_list"]:
        qa["correct"] = results[idx]
        idx += 1

with open(
    "data/annotations/video_list_CZ_answer_clipwise_verified_0320.json", "w"
) as f:
    json.dump(video_list, f, indent=4)

Retry 0 times, exception: Error code: 429 - {'error': {'message': 'qpm limit, you can apply for expansion on the platform (https://gpt.bytedance.net/gpt_openapi/model)', 'code': '-2001'}}
Retry 0 times, exception: Error code: 429 - {'error': {'message': 'qpm limit, you can apply for expansion on the platform (https://gpt.bytedance.net/gpt_openapi/model)', 'code': '-2001'}}
Retry 0 times, exception: Error code: 429 - {'error': {'message': 'qpm limit, you can apply for expansion on the platform (https://gpt.bytedance.net/gpt_openapi/model)', 'code': '-2001'}}
Retry 0 times, exception: Error code: 429 - {'error': {'message': 'qpm limit, you can apply for expansion on the platform (https://gpt.bytedance.net/gpt_openapi/model)', 'code': '-2001'}}
Retry 0 times, exception: Error code: 429 - {'error': {'message': 'qpm limit, you can apply for expansion on the platform (https://gpt.bytedance.net/gpt_openapi/model)', 'code': '-2001'}}
Retry 0 times, exception: Error code: 429 - {'error': {'mess

In [5]:
print(results)

['Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 